In [1]:
# Individual API endpoint tests for notebook use

import httpx
import json
from IPython.display import display, JSON

# Configuration - change this to match your API URL
API_BASE_URL = "http://localhost:8000"

In [2]:
# Test 1: Search Papers
async def test_search_papers(query="reinforcement learning trading", max_results=5):
    """Test the search endpoint"""
    print(f"Testing search endpoint with query: '{query}'")
    
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(
            f"{API_BASE_URL}/search/",
            params={"query": query, "max_results": max_results}
        )
        
        print(f"Status code: {response.status_code}")
        response_data = response.json()
        
        # Display first paper details
        if response.status_code == 200 and "papers" in response_data and len(response_data["papers"]) > 0:
            paper_id = response_data["papers"][0]["id"]
            print(f"Found {len(response_data['papers'])} papers. First paper ID: {paper_id}")
            
            # Return the paper ID for use in other tests
            return paper_id, response_data
        else:
            print("No papers found or error in response")
            return None, response_data
        
results = await test_search_papers(query="momentum trading")

Testing search endpoint with query: 'momentum trading'
Status code: 200
Found 5 papers. First paper ID: http://arxiv.org/abs/2504.10789v1


In [3]:
results

('http://arxiv.org/abs/2504.10789v1',
 {'papers': [{'id': 'http://arxiv.org/abs/2504.10789v1',
    'title': 'Can Large Language Models Trade? Testing Financial Theories with LLM Agents in Market Simulations',
    'authors': ['Alejandro Lopez-Lira'],
    'summary': "This paper presents a realistic simulated stock market where large language\nmodels (LLMs) act as heterogeneous competing trading agents. The open-source\nframework incorporates a persistent order book with market and limit orders,\npartial fills, dividends, and equilibrium clearing alongside agents with varied\nstrategies, information sets, and endowments. Agents submit standardized\ndecisions using structured outputs and function calls while expressing their\nreasoning in natural language. Three findings emerge: First, LLMs demonstrate\nconsistent strategy adherence and can function as value investors, momentum\ntraders, or market makers per their instructions. Second, market dynamics\nexhibit features of real financial ma

In [4]:
# Test 2: Get Paper by ID
async def test_get_paper(paper_id):
    """Test getting a specific paper by ID"""
    print(f"Testing get paper endpoint for ID: {paper_id}")
    
    async with httpx.AsyncClient(timeout=30.0) as client:
        response = await client.get(f"{API_BASE_URL}/search/{paper_id}")
        
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            paper = response.json()
            print(f"Paper title: {paper.get('title', 'N/A')}")
            print(f"Authors: {', '.join(paper.get('authors', []))}")
            return paper
        else:
            print(f"Error fetching paper: {response.text}")
            return None
        
results = await test_get_paper(paper_id="2204.11824")
results

Testing get paper endpoint for ID: 2204.11824
Status code: 200
Paper title: Semi-Parametric Neural Image Synthesis
Authors: Andreas Blattmann, Robin Rombach, Kaan Oktay, Jonas Müller, Björn Ommer


{'id': 'http://arxiv.org/abs/2204.11824v3',
 'title': 'Semi-Parametric Neural Image Synthesis',
 'authors': ['Andreas Blattmann',
  'Robin Rombach',
  'Kaan Oktay',
  'Jonas Müller',
  'Björn Ommer'],
 'summary': 'Novel architectures have recently improved generative image synthesis leading\nto excellent visual quality in various tasks. Much of this success is due to\nthe scalability of these architectures and hence caused by a dramatic increase\nin model complexity and in the computational resources invested in training\nthese models. Our work questions the underlying paradigm of compressing large\ntraining data into ever growing parametric representations. We rather present\nan orthogonal, semi-parametric approach. We complement comparably small\ndiffusion or autoregressive models with a separate image database and a\nretrieval strategy. During training we retrieve a set of nearest neighbors from\nthis external database for each training instance and condition the generative\nmodel o

In [5]:
async def test_paper_structure(paper_id):
    """Test extracting paper structure"""
    print(f"Testing structure endpoint for paper ID: {paper_id}")
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.get(f"{API_BASE_URL}/structure/{paper_id}")
        
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            structure = response.json()
            print("Paper structure sections:")
            for section, content in structure.items():
                print(f"- {section}: {content[:50]}...")
            return structure
        else:
            print(f"Error extracting structure: {response.text}")
            return None
        
results = await test_paper_structure(paper_id="2204.11824")
results

Testing structure endpoint for paper ID: 2204.11824
Status code: 200
Paper structure sections:
- objective: This paper aims to challenge the traditional appro...
- methods: ['Semi-parametric approach combining diffusion or autoregressive models with an external image database', 'Nearest neighbor retrieval strategy for conditioning the generative model during training', 'Post-hoc model transfer by swapping the external database for different contents', 'Evaluation of performance on tasks not explicitly trained on, such as class-conditional synthesis and zero-shot stylization']...
- results: ['Achieved competitive performance on novel tasks without requiring paired text-image data', 'Significantly reduced parameter count of the generative model while outperforming state-of-the-art methods', 'Demonstrated effective transfer of trained models to new domains with different content databases', 'Minimal memory and computational overhead associated with the external database and retrieval proc

{'objective': 'This paper aims to challenge the traditional approach of compressing large training data into complex parametric models by proposing a semi-parametric method that integrates a retrieval strategy with generative models. The goal is to enhance generative image synthesis while reducing model complexity and computational resources.',
 'methods': ['Semi-parametric approach combining diffusion or autoregressive models with an external image database',
  'Nearest neighbor retrieval strategy for conditioning the generative model during training',
  'Post-hoc model transfer by swapping the external database for different contents',
  'Evaluation of performance on tasks not explicitly trained on, such as class-conditional synthesis and zero-shot stylization'],
 'results': ['Achieved competitive performance on novel tasks without requiring paired text-image data',
  'Significantly reduced parameter count of the generative model while outperforming state-of-the-art methods',
  'Demo

In [6]:
# Test 4: Generate Paper Summary
async def test_paper_summary(paper_id):
    """Test generating paper summary"""
    print(f"Testing summary endpoint for paper ID: {paper_id}")
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.get(f"{API_BASE_URL}/summarize/{paper_id}")
        
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            summary = response.json()
            print(f"Summary length: {len(summary.get('summary', ''))}")
            print(f"Keywords: {summary.get('keywords', [])}")
            print(f"Trading relevance: {summary.get('trading_relevance', 'N/A')}")
            return summary
        else:
            print(f"Error generating summary: {response.text}")
            return None

results = await test_paper_summary(paper_id="2204.11824")
results

Testing summary endpoint for paper ID: 2204.11824
Status code: 200
Summary length: 1117
Keywords: ['semi-parametric models', 'generative modeling', 'data retrieval', 'zero-shot learning', 'financial data synthesis', 'adaptive trading strategies', 'market condition adaptation', 'scene composition']
Trading relevance: N/A


{'paper_id': 'http://arxiv.org/abs/2204.11824v3',
 'title': 'Semi-Parametric Neural Image Synthesis',
 'sections': {'objective': 'This paper aims to challenge the traditional approach of compressing large training data into complex parametric models by proposing a semi-parametric method that integrates a retrieval strategy with generative models. The goal is to enhance generative image synthesis while reducing model complexity and computational resources.',
  'methods': ['Semi-parametric approach combining diffusion or autoregressive models with an external image database',
   'Nearest neighbor retrieval strategy for conditioning the generative model during training',
   'Post-hoc model transfer by swapping the external database for different contents',
   'Evaluation of performance on tasks not explicitly trained on, such as class-conditional synthesis and zero-shot stylization'],
  'results': ['Achieved competitive performance on novel tasks without requiring paired text-image data',

In [ ]:
# Test 5: Ask Question
async def test_qa(paper_id, question="What are the main methods used in this paper?"):
    """Test asking a question about a paper"""
    print(f"Testing QA endpoint with question: '{question}'")
    
    async with httpx.AsyncClient(timeout=60.0) as client:
        response = await client.post(
            f"{API_BASE_URL}/qa/",
            json={"question": question, "paper_ids": [paper_id]}
        )
        
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            answer = response.json()
            print(f"Answer: {answer.get('answer', '')[:100]}...")
            print(f"Confidence: {answer.get('confidence', 0)}")
            return answer
        else:
            print(f"Error getting answer: {response.text}")
            return None

In [ ]:
# Test 6: Generate Strategy
async def test_strategy(paper_id):
    """Test generating a trading strategy"""
    print(f"Testing strategy endpoint for paper ID: {paper_id}")
    
    async with httpx.AsyncClient(timeout=120.0) as client:
        response = await client.post(
            f"{API_BASE_URL}/strategy/",
            json={
                "paper_ids": [paper_id],
                "market": "equities",
                "timeframe": "daily",
                "risk_profile": "moderate",
                "additional_context": "Focus on low volatility stocks"
            }
        )
        
        print(f"Status code: {response.status_code}")
        if response.status_code == 200:
            strategy = response.json()
            print(f"Strategy name: {strategy.get('strategy_name', 'N/A')}")
            print(f"Code length: {len(strategy.get('python_code', ''))}")
            return strategy
        else:
            print(f"Error generating strategy: {response.text}")
            return None